# T1036.004 - Masquerade Task or Service
Adversaries may attempt to manipulate the name of a task or service to make it appear legitimate or benign. Tasks/services executed by the Task Scheduler or systemd will typically be given a name and/or description.(Citation: TechNet Schtasks)(Citation: Systemd Service Units) Windows services will have a service name as well as a display name. Many benign tasks and services exist that have commonly associated names. Adversaries may give tasks or services names that are similar or identical to those of legitimate ones.

Tasks or services contain other fields, such as a description, that adversaries may attempt to make appear legitimate.(Citation: Palo Alto Shamoon Nov 2016)(Citation: Fysbis Dr Web Analysis)

## Atomic Tests

### Atomic Test #1 - Creating W32Time similar named service using schtasksCreating W32Time similar named service (win32times) using schtasks just like threat actor dubbed "Operation Wocao"**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
schtasks /create /ru system /sc daily /tr "cmd /c powershell.exe -ep bypass -file c:\T1036.004_NonExistingScript.ps1" /tn win32times /f
schtasks /query /tn win32times
```

In [ ]:
Invoke-AtomicTest T1036.004 -TestNumbers 1

#### Cleanup: 
```cmd
schtasks /tn win32times /delete /f
```

In [ ]:
Invoke-AtomicTest T1036.004 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Creating W32Time similar named service using scCreating W32Time similar named service (win32times) using sc just like threat actor dubbed "Operation Wocao"**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
sc create win32times binPath= "cmd /c start c:\T1036.004_NonExistingScript.ps1"
sc qc win32times
```

In [ ]:
Invoke-AtomicTest T1036.004 -TestNumbers 2

#### Cleanup: 
```cmd
sc delete win32times
```

In [ ]:
Invoke-AtomicTest T1036.004 -TestNumbers 2 -Cleanup

### Atomic Test #3 - linux rename /proc/pid/comm using prctl
Runs a C program that calls prctl(PR_SET_NAME) to modify /proc/pid/comm value to "totally_legit".  This will show up as process name in simple 'ps' listings.

**Supported Platforms:** linux
#### Dependencies:  Run with `sh`!
##### Description: #{exe_path} must be exist on system.

##### Check Prereq Commands:
```sh
stat /tmp/T1036_004_prctl_rename

```
##### Get Prereq Commands:
```sh
cc -o /tmp/T1036_004_prctl_rename PathToAtomicsFolder/T1036.004/src/prctl_rename.c

```

In [ ]:
Invoke-AtomicTest T1036.004 -TestNumbers 3 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
/tmp/T1036_004_prctl_rename & ps
TMP=`ps | grep totally_legit`
if [ -z "${TMP}" ] ; then echo "renamed process NOT FOUND in process list" && exit 1; fi
exit 0
```

In [ ]:
Invoke-AtomicTest T1036.004 -TestNumbers 3

#### Cleanup: 
```sh
rm -f /tmp/T1036_004_prctl_rename
```

In [ ]:
Invoke-AtomicTest T1036.004 -TestNumbers 3 -Cleanup

## Detection
Look for changes to tasks and services that do not correlate with known software, patch cycles, etc. Suspicious program execution through scheduled tasks or services may show up as outlier processes that have not been seen before when compared against historical data. Monitor processes and command-line arguments for actions that could be taken to create tasks or services. Data and events should not be viewed in isolation, but as part of a chain of behavior that could lead to other activities, such as network connections made for Command and Control, learning details about the environment through Discovery, and Lateral Movement.